In [ ]:
import pandas as pd
from datetime import date
import json

 ## appending annotated jsonl batches to get full annotated data

In [ ]:
df_annotated_til_20180219 = pd.read_json('../backend/data/annotations/until_20180219_annotations.jsonl', lines=True)
batches_list = [
    df_annotated_til_20180219
]
df_full_annotations = pd.DataFrame()
for batch in batches_list:
    df_full_annotations = pd.concat([df_full_annotations, batch], ignore_index=True)
df_full_annotations

,date,working,total_points,start_point,end_point
0,2014-06-01,False,1,NaN,NaN
1,2014-06-04,False,1,NaN,NaN
2,2014-06-06,False,13,NaN,NaN
3,2014-06-09,False,1,NaN,NaN
4,2014-06-11,False,1,NaN,NaN
...,...,...,...,...,...
1062,2018-02-15,True,19,0.0,18.0
1063,2018-02-16,True,31,0.0,30.0
1064,2018-02-17,True,30,0.0,29.0
1065,2018-02-18,True,47,0.0,46.0


In [ ]:
df_full_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1067 non-null   datetime64[ns]
 1   working       1067 non-null   bool          
 2   total_points  1067 non-null   int64         
 3   start_point   752 non-null    float64       
 4   end_point     752 non-null    float64       
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1)
memory usage: 34.5 KB


In [ ]:
df_full_annotations.describe().T

,count,mean,min,25%,50%,75%,max,std
date,1067,2016-04-05 23:58:39.025304576,2014-06-01 00:00:00,2015-03-10 12:00:00,2016-05-16 00:00:00,2017-03-08 12:00:00,2018-02-19 00:00:00,NaN
total_points,1067.0,36.374883,1.0,7.5,29.0,57.0,148.0,32.418292
start_point,752.0,5.3125,0.0,0.0,0.0,9.0,62.0,8.38326
end_point,752.0,39.074468,0.0,14.0,34.0,58.0,146.0,30.137056


In [ ]:
df_full_annotations.working.value_counts()

working
True     752
False    315
Name: count, dtype: int64

 ## generating all visits groutes by day

In [ ]:
file = "../backend/data/visitas_business_days_by_day.json"
dict_visits_business_days_by_day = json.load(open(file, 'r'))
dict_visits_business_days_by_day['2018-06-15']

{'data': [{'place': 'Calle 151, Tibabuyes, UPZs de Bogotá, Localidad Suba, Bogotá, Bogotá Distrito Capital - Municipio, Bogotá, Distrito Capital, RAP (Especial) Central, 111161, Colombia',
   'timestamp': '2018-06-15 10:05:00',
   'latitude': 4.7535113,
   'longitude': -74.1021358}],
 'business_day': False}

 ## adding the start and end timestamps to the annotations dataframe

In [ ]:
df_final_contract = df_full_annotations[df_full_annotations['working']].copy()
df_final_contract.reset_index(inplace=True)
df_final_contract['index'] = df_final_contract.index
df_final_contract['start_timestamp'] = None
df_final_contract['end_timestamp'] = None
df_final_contract

,index,date,working,total_points,start_point,end_point,start_timestamp,end_timestamp
0,0,2014-06-16,True,8,3.0,6.0,None,None
1,1,2014-06-17,True,3,0.0,2.0,None,None
2,2,2014-06-19,True,4,0.0,3.0,None,None
3,3,2014-06-20,True,19,1.0,14.0,None,None
4,4,2014-06-21,True,29,0.0,25.0,None,None
...,...,...,...,...,...,...,...,...
747,747,2018-02-15,True,19,0.0,18.0,None,None
748,748,2018-02-16,True,31,0.0,30.0,None,None
749,749,2018-02-17,True,30,0.0,29.0,None,None
750,750,2018-02-18,True,47,0.0,46.0,None,None


In [ ]:
for _, day in df_final_contract.iterrows():
    date_str = str(day.date.strftime('%Y-%m-%d'))
    df_final_contract.loc[day['index'], 'start_timestamp'] = dict_visits_business_days_by_day[date_str]['data'][int(day.start_point)]['timestamp']
    df_final_contract.loc[day['index'], 'end_timestamp'] = dict_visits_business_days_by_day[date_str]['data'][int(day.end_point)]['timestamp']

In [ ]:
df_final_contract = df_final_contract.drop(columns=['index', 'working', 'start_point', 'end_point', 'total_points'])
df_final_contract

,date,start_timestamp,end_timestamp
0,2014-06-16,2014-06-16 11:23:00,2014-06-16 22:10:00
1,2014-06-17,2014-06-17 11:26:00,2014-06-17 22:09:00
2,2014-06-19,2014-06-19 07:23:00,2014-06-19 17:55:00
3,2014-06-20,2014-06-20 07:48:00,2014-06-20 20:11:00
4,2014-06-21,2014-06-21 05:24:00,2014-06-21 16:29:00
...,...,...,...
747,2018-02-15,2018-02-15 08:45:00,2018-02-15 20:28:00
748,2018-02-16,2018-02-16 00:15:00,2018-02-16 22:35:00
749,2018-02-17,2018-02-17 10:34:00,2018-02-17 17:34:00
750,2018-02-18,2018-02-18 20:17:00,2018-02-18 23:24:00


 ## generating final contract

In [ ]:
df_final_contract.to_csv('../backend/data/final_contract.csv', index=False)